<a href="https://colab.research.google.com/github/insaiyancvk/Dere-Net/blob/main/Dere_Net_dirty_work_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. See what data we got

## Plot sample data


In [ ]:
#@title
path = "./waifus/waifus_images/waifus_images_train/Dandere/Kosaki Onodera40.png"
w = cv2.imread(path, 3)
RGB_img = cv2.cvtColor(w, cv2.COLOR_BGR2RGB)
plt.imshow(RGB_img)

In [52]:
#@title
RGB_img.shape, type(RGB_img)

((300, 168, 3), numpy.ndarray)

In [56]:
#@title
TRAIN_PATH

'waifus\\waifus_images\\waifus_images_train'

In [ ]:
#@title
## Print out images
fig, axs = plt.subplots(7,5, figsize=(15,15), constrained_layout=True)
fig = plt.figure(figsize=(10,3))

for ax, row in zip(axs[:,0], class_list):
    ax.set_ylabel(row, rotation=90, fontsize=26)

a, b = 0, 0
for i in os.listdir(TRAIN_PATH):
    imgs = os.listdir(os.path.join(TRAIN_PATH, i))
    random.shuffle(imgs)
    for j in imgs:
        img2 = cv2.imread(os.path.join(TRAIN_PATH, i, j))
        try:
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        except:
            print(os.path.join(TRAIN_PATH, i, j))
            continue
        rm = axs[a][b].imshow(img2)
        plt.axis('off')
        rm.axes.get_xaxis().set_visible(False)
#         rm.axes.get_yaxis().set_visible(False)
        b+=1
        if b>4:
            b=0
            break
    a+=1
    if a>6:
        break

**NOTE:** Use image size: 224,224 or 256,256 ~(since we ain't got chad GPUs)~ we do 😏


## Import stuff

In [18]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os, random

import torch
import torchvision

from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from tqdm import tnrange,tqdm_notebook, tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocess the data

In [13]:
print("Initializing kaggle")
!rm -rf /root/.kaggle
!mkdir /root/.kaggle 
!cp /content/drive/MyDrive/kaggle/kaggle.json /content
!mv kaggle.json /root/.kaggle/kaggle.json

print("Downloading the dataset")
!kaggle datasets download jahelsantiagoleon/female-anime-characters-anime-dataset 
print("Extracting the dataset")
!unzip female-anime-characters-anime-dataset.zip -d .

print("Initializing classes")

class_list = [
    'dandere',
    'deredere',
    'himdere',
    'kundere',
    'tsundere',
    'yandere',
    'yangire'
]

print("Creating folders for workspace")
FOLDERS = {
    "WAIFUS": os.path.join("waifus"),
    # "PROCESSED": os.path.join("waifus","waifus_images"),
    "TRAIN": os.path.join("waifus","waifus_images","waifus_images_train"),
    "TEST": os.path.join("waifus","waifus_images","waifus_images_val"),
    "WORKSPACE": os.path.join("waifus","Workspace"),
    "MODELS" : os.path.join("waifus","Workspace","Models"),
    "LOSS" : os.path.join("waifus","Workspace","Loss"),
    "ACCURACY" : os.path.join("waifus","Workspace","Accuracy"),
    "LOSS_TRAIN" : os.path.join("waifus","Workspace","Loss","Train"),
    "LOSS_TEST" : os.path.join("waifus","Workspace","Loss","Test"),
    "ACCURACY_TRAIN" : os.path.join("waifus","Workspace","Accuracy","Train"),
    "ACCURACY_TEST" :os.path.join("waifus","Workspace","Accuracy","Test")
}

for key in FOLDERS:
    if not os.path.isdir("./"+FOLDERS[key]):
        print(key," directory doesn't exist. Creating one.")
        os.mkdir("./"+FOLDERS[key])
        print(key, " directory created.")
    else:
        print(key, "exists")

print("Transforming the waifu data")
transformations = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transformations_test = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# class ImageFolderWithPaths(torchvision.datasets.ImageFolder):    
#     def __getitem__(self, index):
#         original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
#         path = self.imgs[index][0]
#         tuple_with_path = (original_tuple + (path,))
#         return tuple_with_path

# train_dataset = ImageFolderWithPaths(root="./dataset/waifus/waifus_images/waifus_images_train/", transform = transformations)
print("Creating train dataloader")
train_dataset = datasets.ImageFolder(FOLDERS["TRAIN"], transform = transformations)
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)

# test_dataset = torchvision.datasets.ImageFolder(FOLDERS["TEST"], transform=transformations_test)
print("Creating test dataloader")
test_dataset = datasets.ImageFolder(FOLDERS["TEST"], transform = transformations)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16,
                                         shuffle=True)

print("Data preprocessing completed.")

## The Model:

In [79]:
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.pool = nn.MaxPool2d(5, 5)
        
        self.conv1 = nn.Conv2d(3, 6, 7)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 16, 3)
        
        self.fc1 = nn.Linear(16, 7)
        #self.fc2 = nn.Linear(500, 50)
        #self.fc3 = nn.Linear(50, 7)

    def forward(self, x):
        
        # Conv layer 1
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # Conv layer 2
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # Conv layer 3
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)

        #print("Tensor shape: ",x.shape)
        
        # Flatten the batch
        x = x.view(x.size(0),-1)
        #print(x.shape)
        
        # Dense layer 1
        x = self.fc1(x)
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        return x

In [80]:
net =  Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0005, momentum=0.9)
items = iter(train_loader)

In [81]:
# Testing if all the parameters of the NN are set properly

image, label, p = items.next()
x = net(image[0].unsqueeze(0))
# print(x.shape)
f = plt.imshow(np.transpose(image[0], (1,2,0)));
f.axes.get_xaxis().set_visible(False)
f.axes.get_yaxis().set_visible(False)
plt.title(
    "Name: "+''.join([i for i in os.path.splitext(os.path.basename(p[0]))[0] if not i.isdigit()])+"\n"+
    "Predicted: "+class_list[x.cpu().data.numpy().argmax()]+"\n"+"Actual: "+class_list[label[0].item()+1],
    loc='center'
);

ValueError: ignored

## Steps
- pass imgae to the NN
- calculate loss
- backprop
- update weights

In [82]:
def train_step(model, images, labels, loss_func, optim, device):
    
    # Set the gradiant values to zero because torch accumulates the values
    optim.zero_grad()
    
    # Predictions from the model
    outputs = model(images.to(device))
    
    # Calculate the loss
    loss = loss_func(outputs, labels.to(device))
    
    # Finds the derivative by applying chain rule
    loss.backward()
    
    # Move the loss function towards the local/global minima
    optim.step()
    
    # return the outputs and loss by moving them to CPU
    return outputs.cpu().detach().numpy(), loss.cpu().item()

In [83]:
def epoch(model, train_loader, loss_func, optim, device):
    mean_loss = 0
    model = model.to(device)
    for image, label in tqdm(train_loader):
        loss = train_step(model, image, label, loss_func, optim, device)
        mean_loss += loss/len(train_loader)
    return mean_loss

5. Build a training loop
    1. Move to GPU
    2. Define the training function
    3. Calculate the running loss and store the same in list and later convert to np array and save the np array.
    4. Calculate the accuracy after every batch. Calculate the average of same and store the values in a list 
       and later convert it to np array and save theh np array.
    5. After every epoch, test the model loss and accuracy 
       (by following the same steps as done for training ie, move to list, convert to np array, save the np array) with the test data.


In [98]:
def epoch(model, test_loader ,train_loader, loss_func, optim, device):

   train_loss_vals = []
   train_accuracy_vals = []
   test_loss_vals = []
   test_accuracy_vals = []

   train_batch_loss = 0
   train_batch_accuracy = 0

   model = model.to(device)

   print("\n\t\tTraining the batch... \n")

   with tqdm(len(train_loader), position=0, leave=True) as pbar:
    for image, label in tqdm(train_loader, ncols=100, position=0, leave=True):

        optim.zero_grad()
        
        output = model(image.to(device))
        
        loss = loss_func(
          output,
          label.to(device)
          )
        
        loss.backward()
        optim.step()

        accuracy = np.average(
              np.argmax(
                output.cpu().detach().numpy(), axis=1) == label.cpu().detach().numpy()
          )

        train_loss_vals.append(loss.cpu().item()) # training loss for graph
        train_batch_loss += loss/len(train_loader) # training average loss
        train_accuracy_vals.append(accuracy) # training accuray for graph
        train_batch_accuracy += accuracy/len(train_loader) # training average accuracy
        pbar.update()
   
   print("\n\t\tValidating the batch... \n")

   with tqdm(len(test_loader), position=0, leave=True) as pbar:
    for image, label in tqdm(test_loader, ncols=100, position=0, leave=True):

        output = model(image.to(device))

        loss = loss_func(
          output,
          label.to(device)
        )

        accuracy = np.average(
          np.argmax(
              output.cpu().detach().numpy(), axis=1) == label.cpu().detach().numpy()
          )

        test_loss_vals.append(loss.cpu().item()) # testing loss for graph
        test_accuracy_vals.append(accuracy) # testing accuracy for graph
        pbar.update()

   return (

      # TRAIN LOSS
      train_loss_vals, # list
      train_batch_loss, # float

      # TRAIN ACCURACY
      train_accuracy_vals, # list
      train_batch_accuracy, # float

      # TEST LOSS, ACCURACY
      test_loss_vals, # list
      test_accuracy_vals # list
   )

In [92]:
net =  Net()
net = net.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0005, momentum=0.9)

In [90]:
preds.shape, label.cpu().detach().numpy().shape

((16, 7), (16,))

In [94]:
import os, json, warnings
warnings.filterwarnings("ignore")

In [99]:
vals = epoch(net, test_loader ,train_loader, loss_func, optimizer, device)

  0%|                                                               | 1/871 [00:00<01:47,  8.09it/s]


		Training the batch... 



100%|█████████████████████████████████████████████████████████████| 871/871 [01:49<00:00,  7.92it/s]
871it [01:49,  7.92it/s]
  0%|                                                                       | 0/168 [00:00<?, ?it/s]
0it [00:00, ?it/s]


		Validating the batch... 



NameError: ignored

In [ ]:
res = net(image[0].unsqueeze(0).cuda())

In [ ]:
res

tensor([[-0.1173, -0.1199,  0.1329,  0.2926,  0.3461, -0.1499,  0.1409]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [ ]:
label[0]

tensor(4)

In [ ]:
res.cpu().data.numpy().argmax()

4

In [ ]:
# use python list to store loss and later convert it to np array and delete python list

pyloss = []
for i in range(10):
    pyloss.append(i)
nploss = np.array(pyloss, dtype="float64")
print("pyloss before del: ", pyloss)
del pyloss
print(nploss)

pyloss before del:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [ ]:
x

tensor([[ 0.2325,  0.0674, -0.0641, -0.0441,  0.1456,  0.1779,  0.2395]],
       grad_fn=<AddmmBackward>)

In [ ]:
tes = [1 if net(image[i].unsqueeze(0).cuda()).cpu().data.numpy().argmax() == label[i] else 0 for i in range(10)]

In [ ]:
net.state_dict()

In [ ]:
optimizer.state_dict()

{'state': {0: {'momentum_buffer': tensor([[[[ 4.1772e-02,  4.6386e-02,  3.7139e-02,  2.8831e-02,  1.2046e-02,
               1.4159e-03,  1.7098e-03],
             [ 4.1783e-02,  4.1015e-02,  3.0438e-02,  1.6739e-02,  7.2340e-04,
              -3.8403e-03,  2.9342e-03],
             [ 3.3879e-02,  3.0395e-02,  2.2365e-02,  9.1122e-03,  6.5250e-04,
               4.7763e-03,  1.4419e-02],
             [ 3.0136e-02,  2.6518e-02,  1.7585e-02,  4.4089e-03,  8.9547e-04,
               2.3401e-03,  1.6563e-02],
             [ 3.0620e-02,  2.1740e-02,  7.7112e-03,  8.2436e-04, -4.4799e-03,
              -2.3937e-03,  1.4382e-02],
             [ 2.3280e-02,  1.6441e-02,  3.6832e-03, -2.9148e-03, -5.4497e-03,
              -5.6341e-04,  1.4410e-02],
             [ 1.2070e-02,  1.1348e-02,  7.1764e-03,  3.2737e-03,  6.9680e-04,
               3.9707e-03,  1.2335e-02]],
   
            [[ 4.8708e-02,  4.9983e-02,  3.8166e-02,  2.8286e-02,  1.0263e-02,
              -3.5417e-03, -4.7380e-03],
    

In [ ]:
EPOCH = 1
PATH = "./dataset/waifus/models/"
MODEL1 = "3conv_3RLU_3MP2d_1d_crossentropy_sgd.pt"
LOSS = 1.9

In [ ]:
# Save the model
torch.save({
    'epoch': EPOCH,
    'model_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': LOSS
}, 
    PATH+MODEL1)

In [ ]:
# Load the saved model
checkpoint = torch.load(PATH+MODEL1)
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])